In [ ]:
#Decrição: usando indicadores de acao com machine learning para tentar prever a direcao do preço de uma acao


In [2]:
#Importando as bibliotecas
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [3]:
#load the data set

import pandas_datareader.data as web # p importar dados do yahoo
!pip install yfinance --upgrade --no-cache-dir # para puxar dados do yahoo
import yfinance as yf
df = web.get_data_yahoo('^BVSP') #ticket do ativo


     |████████████████████████████████| 5.5MB 7.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=6dc550e2646e07414b4558ff5c3d65fb22dc5c5c848ee8510e56c6183f685896
  Stored in directory: /tmp/pip-ephem-wheel-cache-st7shcle/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [11]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-02-22,43345.0,41544.0,41544.0,43235.0,4508500.0,43235.0
2016-02-23,43601.0,42359.0,43235.0,42521.0,3805300.0,42521.0
2016-02-24,42521.0,41211.0,42521.0,42085.0,3522700.0,42085.0
2016-02-25,42327.0,41442.0,42084.0,41888.0,4046700.0,41888.0
2016-02-26,42495.0,41417.0,41889.0,41593.0,3274600.0,41593.0
...,...,...,...,...,...,...
2021-02-09,119975.0,118246.0,119696.0,119429.0,7913500.0,119429.0
2021-02-10,119738.0,117970.0,119472.0,118430.0,8644600.0,118430.0
2021-02-11,120283.0,118440.0,118440.0,119235.0,7567400.0,119235.0


In [4]:
# criando funcoes p calcular SMA(media movel simples) e  EMA(media movel exponencial)
# tempos comuns para essas media 15, 20 e 30 

def SMA(data,period=30,column='Close'):
  return data[column].rolling(window=period).mean()

def EMA(data,period=20,column='Close'):
  return data[column].ewm(span=period,adjust=False).mean()



In [6]:
#criando uma funcao para calcular a convergencia e divergencia entre as medias moveis (MACD)
def MACD(data,period_long=26,period_short=12,period_signal=9,column='Close'):
  #calcula the short EMA
  shortEMA = EMA(data, period=period_short, column = column) 
  #calculando the long EMA
  longEMA = EMA(data,period=period_long, column=column)
  #calculando e salvando MACD
  data['MACD'] = shortEMA - longEMA
  #calculando o linha de sinal e salvando no dataframe
  data['signal_Line'] = EMA(data, period=period_signal, column='MACD')
  return data


In [7]:
#criando uma funcao para calcular o Relative Strength(RSI)
def RSI(data, period=14, column='Close'):
  delta = data[column].diff(1)
  delta = delta.dropna()
  up = delta.copy()
  down = delta.copy()
  up[up <0] = 0 
  down[down > 0] = 0
  data['up'] = up
  data['down'] = down
  AVG_Gain = SMA(data, period, column='up')
  AVG_Loss = abs(SMA(data, period, column='down'))
  RS = AVG_Gain/AVG_Loss
  RSI = 100.0 - (100.0/(1.0 + RS) )
  data['RSI'] = RSI
  return data
  


In [8]:
# add os indicadores no data set
MACD(df)
RSI(df)
df['SMA'] = SMA(df)
df['EMA'] = EMA(df)

df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_Line,up,down,RSI,SMA,EMA
Date,,,,,,,,,,,,,
2016-02-22,43345.000000,41544.000000,41544.000000,43235.00000,4508500.0,43235.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,43235.000000
2016-02-23,43601.000000,42359.000000,43235.000000,42521.00000,3805300.0,42521.00000,-56.957265,-11.391453,0.0,-714.00000,NaN,NaN,43167.000000
2016-02-24,42521.000000,41211.000000,42521.000000,42085.00000,3522700.0,42085.00000,-135.713444,-36.255851,0.0,-436.00000,NaN,NaN,43063.952381
2016-02-25,42327.000000,41442.000000,42084.000000,41888.00000,4046700.0,41888.00000,-211.585511,-71.321783,0.0,-197.00000,NaN,NaN,42951.956916
2016-02-26,42495.000000,41417.000000,41889.000000,41593.00000,3274600.0,41593.00000,-292.150970,-115.487620,0.0,-295.00000,NaN,NaN,42822.532448
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,119738.000000,117970.000000,119472.000000,118430.00000,8644600.0,118430.00000,358.647535,523.643208,0.0,-999.00000,46.220277,119737.166667,118856.959588
2021-02-11,120283.000000,118440.000000,118440.000000,119235.00000,7567400.0,119235.00000,356.316118,490.177790,805.0,0.00000,52.407880,119743.300000,118892.963436
2021-02-12,119763.000000,118163.000000,119300.000000,119116.00000,7655900.0,119116.00000,340.936050,460.329442,0.0,-119.00000,56.355434,119731.333333,118914.205014


In [9]:
#create the target column
df['Target'] = np.where(df['Close'].shift(-1) > df['Close'], 1, 0)#checando se o preco de amanha é maior do q o de hj e se sim coloca 0
df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_Line,up,down,RSI,SMA,EMA,Target
Date,,,,,,,,,,,,,,
2016-02-22,43345.000000,41544.000000,41544.000000,43235.00000,4508500.0,43235.00000,0.000000,0.000000,NaN,NaN,NaN,NaN,43235.000000,0
2016-02-23,43601.000000,42359.000000,43235.000000,42521.00000,3805300.0,42521.00000,-56.957265,-11.391453,0.0,-714.00000,NaN,NaN,43167.000000,0
2016-02-24,42521.000000,41211.000000,42521.000000,42085.00000,3522700.0,42085.00000,-135.713444,-36.255851,0.0,-436.00000,NaN,NaN,43063.952381,0
2016-02-25,42327.000000,41442.000000,42084.000000,41888.00000,4046700.0,41888.00000,-211.585511,-71.321783,0.0,-197.00000,NaN,NaN,42951.956916,0
2016-02-26,42495.000000,41417.000000,41889.000000,41593.00000,3274600.0,41593.00000,-292.150970,-115.487620,0.0,-295.00000,NaN,NaN,42822.532448,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,119738.000000,117970.000000,119472.000000,118430.00000,8644600.0,118430.00000,358.647535,523.643208,0.0,-999.00000,46.220277,119737.166667,118856.959588,1
2021-02-11,120283.000000,118440.000000,118440.000000,119235.00000,7567400.0,119235.00000,356.316118,490.177790,805.0,0.00000,52.407880,119743.300000,118892.963436,0
2021-02-12,119763.000000,118163.000000,119300.000000,119116.00000,7655900.0,119116.00000,340.936050,460.329442,0.0,-119.00000,56.355434,119731.333333,118914.205014,1


In [10]:
#Removendo os 29 primeiros dias do dado
df = df[29:]
df

,High,Low,Open,Close,Volume,Adj Close,MACD,signal_Line,up,down,RSI,SMA,EMA,Target
Date,,,,,,,,,,,,,,
2016-04-04,50556.000000,48600.000000,50556.000000,48780.00000,3556500.0,48780.00000,1235.145790,1505.289640,0.0,-1782.00000,49.653855,47842.966667,49226.808556,1
2016-04-05,49629.000000,48149.000000,48778.000000,49054.00000,3546100.0,49054.00000,1075.317241,1419.295160,274.0,0.00000,58.663545,48036.933333,49210.350598,0
2016-04-06,49054.000000,47874.000000,49054.000000,48096.00000,3810000.0,48096.00000,861.419463,1307.720021,0.0,-958.00000,51.456820,48222.766667,49104.221970,1
2016-04-07,48940.000000,48099.000000,48099.000000,48513.00000,3059900.0,48513.00000,717.283945,1189.632806,417.0,0.00000,36.193214,48437.033333,49047.915115,1
2016-04-08,50486.000000,48517.000000,48517.000000,50293.00000,5047700.0,50293.00000,738.177384,1099.341721,1780.0,0.00000,47.484580,48717.200000,49166.494628,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10,119738.000000,117970.000000,119472.000000,118430.00000,8644600.0,118430.00000,358.647535,523.643208,0.0,-999.00000,46.220277,119737.166667,118856.959588,1
2021-02-11,120283.000000,118440.000000,118440.000000,119235.00000,7567400.0,119235.00000,356.316118,490.177790,805.0,0.00000,52.407880,119743.300000,118892.963436,0
2021-02-12,119763.000000,118163.000000,119300.000000,119116.00000,7655900.0,119116.00000,340.936050,460.329442,0.0,-119.00000,56.355434,119731.333333,118914.205014,1


In [14]:
#split o dataset into a feature or independent data set (X) and a target or dependent data set (Y)
keep_columns = ['Close','MACD','signal_Line','RSI','SMA','EMA']
X = df[keep_columns].values
Y = df['Target'].values

In [15]:
#dividindo o dado de novo, mas dessa vez 80% training e 20% testando o data set
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state = 2)


In [16]:
#criando e treinando a decision tree classifier model
tree = DecisionTreeClassifier().fit(X_train, Y_train)

In [17]:
#checando quao bem o modelo de deu no trading data set
print(tree.score(X_train, Y_train))

1.0


In [18]:
#checando quao bem o modelo de deu no trading data set
print(tree.score(X_test, Y_test))

0.49173553719008267


In [19]:
# mostrando o modelo tree predicitions
tree_preditions = tree.predict(X_test)
print(tree_preditions)


[0 1 1 1 1 1 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 1 0 1 0 1 0 0 1 1 1 1 0 1
 1 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 1 1
 1 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 1 1 0 1 0 0 0
 1 0 1 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 1
 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 0 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 1 0 1
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0 0 1 1 1 0]


In [20]:
#mostrando the actual values from the test data
Y_test

array([0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1])

In [21]:
#pegando as metricas do modelo
from sklearn.metrics import classification_report
print(classification_report(Y_test, tree_preditions))

              precision    recall  f1-score   support

           0       0.42      0.46      0.44       104
           1       0.56      0.51      0.54       138

    accuracy                           0.49       242
   macro avg       0.49      0.49      0.49       242
weighted avg       0.50      0.49      0.49       242

